# TensorFlow Mechanics 101

#### Changes Chong Min made

* Optimizer: from Gradient Descent to Adam
* batch size: set to 10
* size of hidden layer: 8
* weight initializer: random_normal_initializer
* max steps: 20000

Among the above changes, `batch size`, `optimizer`, and `max steps` values were critical.

Because batches were randomly generated, the accuracies were flucturated. It should be fixed.

- This tutorial is meant as a companion to the code [here](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/tutorials/mnist/)
- The goal of this tutorial is to show how to use TensorFlow to train and evaluate a simple feed-forward neural network for handwritten digit classification using the (classic) MNIST data set. 

- [`mnist.py`](https://www.tensorflow.org/code/tensorflow/examples/tutorials/mnist/mnist.py), the code for making a fully-connected MNIST model
- [`fully_connected_feed.py`](https://www.tensorflow.org/code/tensorflow/examples/tutorials/mnist/fully_connected_feed.py), the main code to train the built MNIST model against the downloaded dataset using a feed dictionary.

In [3]:
import math
import time
import random
from os import getcwd
from os.path import join

import pandas as pd
import tensorflow as tf
import numpy as np

## Using a Custom e-rater Data-set

In [4]:
# Download test_data_revised.zip (in email since it can't be shared) and
# save it somewhere, preferably in the directory in which this notebook is
# stored. If it is somewhere else, just make sure to pass in the path when
# this function is used.

def read_praxis_data(macro_or_micro="macro",
                     dev_set=True,
                     data_path=join(getcwd(), "test_data_revised")):
    """
    Read in data from a directory with the following files:
    testing_macro.csv and training_macro.csv.

    Either read in the "macro" files or the "micro" files.

    Returns 1) training IDs, features, and labels,
            2) test IDs, features, and labels, and
            3) development set IDs, features, and labels (if `dev_set` is
               False, the test test set will contain all data originally in
               the "testing" file).
    """
    
    train_data_path = join(data_path,
                           "training_{}.csv"
                           .format(macro_or_micro if macro_or_micro == "macro"
                                   else "micro_revised"))
    train_data = pd.read_csv(train_data_path, dtype={'appointment_id': str})
    train_labels = train_data['H1'].apply(lambda x: x - 1)
    train_features = train_data[[a for a in train_data.columns
                                 if a not in ['appointment_id', 'H1']]]
    train_ids = train_data['appointment_id']

    test_data_path = join(data_path,
                          "testing_{}.csv"
                          .format(macro_or_micro if macro_or_micro == "macro"
                                  else "micro_revised"))
    test_data = pd.read_csv(test_data_path, dtype={'appointment_id': str})
     
    test_labels = test_data['H1'].apply(lambda x: x - 1)
    test_features = test_data[[a for a in test_data.columns
                               if a not in ['appointment_id', 'H1']]]
    test_ids = test_data['appointment_id']

    dev_features = None
    dev_labels = None
    dev_ids = None
    if dev_set:
        test_amount = len(test_features) - 1000
        dev_features = test_features.head(1000)
        dev_features.index = range(len(dev_features))
        test_features = test_features.tail(test_amount)
        test_features.index = range(len(test_features))
        dev_labels = test_labels.head(1000)
        dev_labels.index = range(len(dev_labels))
        test_labels = test_labels.tail(test_amount)
        test_labels.index = range(len(test_labels))
        dev_ids = test_ids[:1000]
        test_ids = test_ids[1000:]

    return (train_ids,
            train_features,
            train_labels,
            test_ids,
            test_features,
            test_labels,
            dev_ids,
            dev_features,
            dev_labels)

In [5]:
# Choose "micro" or "macro". This will change the types of features we're
# using. There are 220 "micro" features in total while thre are 9 macro
# features.
dataset_type = "macro"
# dataset_type = "micro"

In [6]:
# Read in data
(train_ids, train_features, train_labels,
 test_ids, test_features, test_labels,
 dev_ids, dev_features, dev_labels) = read_praxis_data(macro_or_micro=dataset_type,
                                                       dev_set=False)

### Data and Data Shape

In [7]:
# print("Shape of data:\n\tTraining: {}\n\t{}Test: {}"
#       .format(train_features.shape,
#               "" if dev_features is None
#                  else "Development: {}\n\t".format(dev_features.shape),
#               test_features.shape))
# print("Shape of labels data:\n\tTraining: {}\n\t{}Test: {}"
#       .format(train_labels.shape,
#               "" if dev_labels is None
#                  else "Development: {}\n\t".format(dev_labels.shape),
#               test_labels.shape))

In [8]:
# What the features look like
# train_features.head()

In [9]:
# test_features.head()

In [10]:
# Labels are on a 0 to 5 scale (scores 1 to 6)
# train_labels[:10]

In [11]:
# train_labels = np.array(train_labels, dtype=np.float32)
# test_labels = np.array(test_labels, dtype=np.float32)

In [12]:
# if dev_labels is not None:
#     print(dev_labels.value_counts())

In [29]:
# Define some parameters
log_dir_path = join(getcwd(), "logs")
max_steps = 20000
if dataset_type == "macro":
    learning_rate = 0.01
    hidden1 = 8  # 128
    hidden2 = 8  # 32
    hidden3 = 8  # 16
    NUM_FEATURES = 9
    batch_size = 10
else:
    learning_rate = 0.01
    hidden1 = 512
    hidden2 = 128
    hidden3 = 16
    NUM_FEATURES = 220
    batch_size = 200
NUM_CLASSES = 6

## Functions Based on `mnist.py`

In [33]:
def inference(inputs, num_features, num_classes, hidden1_units,
              hidden2_units, hidden3_units):
    """
    Build a model on the inputs up to where it may be used for
    inference.

    Args:
        inputs: Placeholder for input data samples.
        num_features: Number of features in input data.
        num_classes: Number of classes/score labels.
        hidden1_units: Size of the first hidden layer.
        hidden2_units: Size of the second hidden layer.
        hidden3_units: Size of the third hidden layer.

    Returns:
        softmax_linear: Output tensor with the computed logits.
    """

    # Hidden 1
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
              tf.random_normal_initializer(0.0, 0.05)([num_features, hidden1_units]),
              name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]),
                             name='biases')
        hidden1 = tf.nn.relu(tf.matmul(inputs, weights) + biases)

#     # Hidden 2
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
              tf.random_normal_initializer(0.0, 0.05)([hidden1_units, hidden2_units]),
              name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]),
                             name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)

#         weights = tf.Variable(
#             tf.truncated_normal([hidden1_units, hidden2_units],
#                                 stddev=1.0 / math.sqrt(float(hidden1_units))),
#             name='weights')
#         biases = tf.Variable(tf.zeros([hidden2_units]),
#                              name='biases')
#         hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)

#     # Hidden 3
#     with tf.name_scope('hidden3'):
#         weights = tf.Variable(
#             tf.truncated_normal([hidden2_units, hidden3_units],
#                                 stddev=1.0 / math.sqrt(float(hidden2_units))),
#             name='weights')
#         biases = tf.Variable(tf.zeros([hidden3_units]),
#                              name='biases')
#         hidden3 = tf.nn.relu(tf.matmul(hidden2, weights) + biases)

#     # Linear
#     with tf.name_scope('softmax_linear'):
#         weights = tf.Variable(
#             tf.truncated_normal([hidden3_units, num_classes],
#                                 stddev=1.0 / math.sqrt(float(hidden3_units))),
#             name='weights')
#         biases = tf.Variable(tf.zeros([num_classes]),
#                              name='biases')
#         logits = tf.matmul(hidden3, weights) + biases
        
    # Linear
    with tf.name_scope('softmax_linear'):
        
        weights = tf.Variable(
              tf.random_normal_initializer(0.0, 0.05)([hidden1_units, num_classes]),
              name='weights')
        biases = tf.Variable(tf.zeros([num_classes]),
                             name='biases')
        logits = tf.matmul(hidden2, weights) + biases
#         weights = tf.Variable(
#             tf.truncated_normal([hidden3_units, num_classes],
#                                 stddev=1.0 / math.sqrt(float(hidden3_units))),
#             name='weights')
#         biases = tf.Variable(tf.zeros([num_classes]),
#                              name='biases')
#         logits = tf.matmul(hidden3, weights) + biases

    return logits


def loss(logits, labels):
    """
    Calculates the loss from the logits and the labels.

    Args:
        logits: Logits tensor, float - [batch_size, NUM_CLASSES].
        labels: Labels tensor, int32 - [batch_size].

    Returns:
        loss: Loss tensor of type float.
    """

    labels = tf.to_int64(labels)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
          labels=labels, logits=logits, name='xentropy')
    return tf.reduce_mean(cross_entropy, name='xentropy_mean')    

#     cross_entropy = \
#         tf.nn.softmax_cross_entropy_with_logits(labels=labels,
#                                                        logits=logits,
#                                                        name='xentropy')

#     return tf.reduce_mean(cross_entropy, name='xentropy_mean')

#     labels = tf.cast(labels, tf.float32)
#     labels = tf.to_float32(labels)
#     return tf.nn.l2_loss(tf.squared_difference(tf.argmax(logits, 1), labels))  # tf.reduce_mean(tf.square(tf.sub(labels, tf.argmax(logits, 1))))


def training(_loss, learning_rate):
    """
    Sets up the training Ops.
    
    Creates a summarizer to track the loss over time in TensorBoard.
    Creates an optimizer and applies the gradients to all trainable
    variables.
    
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train.

    Args:
        _loss: Loss tensor, from loss().
        learning_rate: The learning rate to use for gradient descent.

    Returns:
        train_op: The Op for training.
    """

    # Add a scalar summary for the snapshot loss.
    tf.summary.scalar('loss', _loss)

    # Create the gradient descent optimizer with the given learning
    # rate.
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) # tf.train.GradientDescentOptimizer(learning_rate)

    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single
    # training step.
    train_op = optimizer.minimize(_loss, global_step=global_step)

    return train_op


def evaluation(logits, labels):
    """
    Evaluate the quality of the logits at predicting the label.

    Args:
        logits: Logits tensor, float - [batch_size, NUM_CLASSES].
        labels: Labels tensor, int32 - [batch_size], with values in the
                range [0, NUM_CLASSES).

    Returns:
        A scalar int32 tensor with the number of examples (out of
        batch_size) that were predicted correctly.
    """

    # For a classifier model, we can use the in_top_k Op.
    # It returns a bool tensor with shape [batch_size] that is true for
    # the examples where the label is in the top k (here k=1)
    # of all logits for that example.
    correct = tf.nn.in_top_k(logits, labels, 1)

#     Return the number of true entries.
    return tf.reduce_sum(tf.cast(correct, tf.int32))

#     return tf.sqrt(tf.reduce_mean(tf.square(tf.sub(logits, labels))))

In [34]:
# From fully_connected_feed.py
def fill_feed_dict(features, labels, inputs_pl, labels_pl, batch_size):
    """
    Fills the feed_dict for training the given step.

    A feed_dict takes the form of:
    feed_dict = {
        <placeholder>: <tensor of values to be passed for placeholder>,
        ....
    }

    Args:
        data_set: The set of features and labels.
        inputs_pl: The input data placeholder.
        labels_pl: The input labels placeholder.
        batch_size: Size of each randomly-selected batch.

    Returns:
        feed_dict: The feed dictionary mapping from placeholders to values.
    """

    # Create the feed_dict for the placeholders filled with the next
    # `batch_size` samples.
    sample = random.sample(range(len(labels)), k=batch_size)
    inputs_feed = features.ix[sample]
    labels_feed = labels.ix[sample]

    feed_dict = {
        inputs_pl: inputs_feed,
        labels_pl: labels_feed,
    }

    return feed_dict


def do_eval(sess, eval_correct, inputs_placeholder, labels_placeholder,
            features, labels):
    """
    Runs one evaluation against the full epoch of data.

    Args:
        sess: The session in which the model has been trained.
        eval_correct: The Tensor that returns the number of correct
                      predictions.
        inputs_placeholder: The input data placeholder.
        labels_placeholder: The labels placeholder.
        data_set: The set of images and labels to evaluate, from
                  input_data.read_data_sets().
    """

    # And run one epoch of eval.
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = features.shape[0] // batch_size
    num_examples = steps_per_epoch * batch_size
    for step in range(steps_per_epoch):
        feed_dict = fill_feed_dict(features, labels,
                                   inputs_placeholder,
                                   labels_placeholder,
                                   batch_size)
        logit_output, true_cnt = sess.run([logits, eval_correct],
                                          feed_dict=feed_dict)
        true_count += true_cnt

    precision = float(true_count) / num_examples
    print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
          (num_examples, true_count, precision))

In [35]:
# Tell TensorFlow that the model will be built into the default Graph.
with tf.Graph().as_default():

    # Generate placeholders for the input feature data and labels.
    inputs_placeholder = tf.placeholder(tf.float32, shape=(batch_size,
                                                           NUM_FEATURES))
    labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))

    # Build a Graph that computes predictions from the inference model.
    logits = inference(inputs_placeholder,
                       NUM_FEATURES,
                       NUM_CLASSES,
                       hidden1,
                       hidden2,
                       hidden3)

    # Add to the Graph the Ops for loss calculation.
    loss_ = loss(logits, labels_placeholder)

    # Add to the Graph the Ops that calculate and apply gradients.
    train_op = training(loss_, learning_rate)

    # Add the Op to compare the logits to the labels during evaluation.
    eval_correct = evaluation(logits, labels_placeholder)

    # Build the summary Tensor based on the TF collection of Summaries.
    summary = tf.summary.merge_all()

    # Add the variable initializer Op.
    init = tf.global_variables_initializer()

    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver()

    # Create a session for running Ops on the Graph.
    sess = tf.Session()

    # Instantiate a SummaryWriter to output summaries and the Graph.
    summary_writer = tf.summary.FileWriter(log_dir_path, sess.graph)

    # And then after everything is built:

    # Run the Op to initialize the variables.
    sess.run(init)

    # Start the training loop.
    for step in range(max_steps):
        start_time = time.time()

        # Fill a feed dictionary with the actual set of images and labels
        # for this particular training step.
        feed_dict = fill_feed_dict(train_features, train_labels,
                                   inputs_placeholder,
                                   labels_placeholder,
                                   batch_size)

        # Run one step of the model.  The return values are the activations
        # from the `train_op` (which is discarded) and the `loss` Op.  To
        # inspect the values of your Ops or variables, you may include them
        # in the list passed to sess.run() and the value tensors will be
        # returned in the tuple from the call.
        _, loss_value = sess.run([train_op, loss_],
                                 feed_dict=feed_dict)

        duration = time.time() - start_time

        # Write the summaries and print an overview fairly often.
        if step % 100 == 0:

            # Print status to stdout.
            print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
            # Update the events file.
            summary_str = sess.run(summary, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()
        
        # Save a checkpoint and evaluate the model periodically.
        if (step + 1) % 1000 == 0 or (step + 1) == max_steps:
            checkpoint_file = join(log_dir_path, 'model.ckpt')
            saver.save(sess, checkpoint_file, global_step=step)

            # Evaluate against the training set.
            print('Train Data Eval:')
            do_eval(sess,
                    eval_correct,
                    inputs_placeholder,
                    labels_placeholder,
                    train_features, train_labels)

            # Evaluate against the development set.
            if dev_labels is not None:
                print('Development Data Eval:')
                do_eval(sess,
                        eval_correct,
                        inputs_placeholder,
                        labels_placeholder,
                        dev_features,
                        dev_labels)

            # Evaluate against the test set.
            print('Test Data Eval:')
            do_eval(sess,
                    eval_correct,
                    inputs_placeholder,
                    labels_placeholder,
                    test_features, test_labels)

Step 0: loss = 1.80 (0.009 sec)
Step 100: loss = 1.22 (0.001 sec)
Step 200: loss = 1.13 (0.001 sec)
Step 300: loss = 1.37 (0.001 sec)
Step 400: loss = 1.10 (0.001 sec)
Step 500: loss = 1.28 (0.001 sec)
Step 600: loss = 0.90 (0.001 sec)
Step 700: loss = 1.08 (0.002 sec)
Step 800: loss = 0.93 (0.002 sec)
Step 900: loss = 1.14 (0.002 sec)
Train Data Eval:
  Num examples: 4000  Num correct: 2058  Precision @ 1: 0.5145
Test Data Eval:
  Num examples: 2750  Num correct: 1462  Precision @ 1: 0.5316
Step 1000: loss = 1.37 (0.002 sec)
Step 1100: loss = 1.27 (0.002 sec)
Step 1200: loss = 0.87 (0.001 sec)
Step 1300: loss = 0.96 (0.001 sec)
Step 1400: loss = 0.65 (0.001 sec)
Step 1500: loss = 1.11 (0.002 sec)
Step 1600: loss = 0.86 (0.001 sec)
Step 1700: loss = 1.13 (0.002 sec)
Step 1800: loss = 1.34 (0.001 sec)
Step 1900: loss = 1.01 (0.001 sec)
Train Data Eval:
  Num examples: 4000  Num correct: 2538  Precision @ 1: 0.6345
Test Data Eval:
  Num examples: 2750  Num correct: 1706  Precision @ 1: 0